In [19]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
from __future__ import print_function
import json

from datetime import date
import datetime
#imports

There are 20 teams in the PL, each plays 38 matches for a total of 760. The first game in the 2022-2023 season is number 18202, and it counts up one by one from there, so the total range will be 18,202 to 18,961. Then we will take only the completed games from there.

In [20]:
table_url = 'https://theathletic.com/football/premier-league/standings/'

In [21]:
res = requests.get(table_url)
soup = BeautifulSoup(res.content, 'lxml')
table_data = soup.find_all('div', {"class": "sc-c8ee3952-0 sc-83a99c6-6 cOGVhH jOibkY"})
#Getting data for number of total matches

In [22]:
result = table_data[::8]
#The "Matches Played" column is every 9th row so that's what we want -- then slice them so they're only the numbers and sum them

In [23]:
result = [str(i) for i in result]
#Turn soup element into string
result = [result[i].split('</')[0] for i in range(len(result))]
#Then split it and take the element with the number
result = [result[i].split('>')[1] for i in range(len(result))]
#Then split it again and take only the number itself
result = [int(i) for i in result]
#Cast the numbers as integers
total_matches = int(sum(result) / 2)
#Sum elements and divide by two for number of matches

In [24]:
total_matches

116

In [25]:
# d = range(18202,18962)
match_range = range(18202,18240)

In [26]:
urls = []
for i in match_range:
    base_url = f'https://understat.com/match/{i}'
    urls.append(base_url)

In [27]:
all_res_data = []
for i in urls:
    res = requests.get(i)
    soup = BeautifulSoup(res.content, 'lxml')
    scripts = soup.find_all('script')
#     positions = soup.find_all("span", {"class": "player-position"})

    all_res_data.append(scripts)
    #Here we can append only the ones that have 'script'

In [28]:
backup_all_res_data = all_res_data
#Adding a backup unless we need it and so we don't need to scrape again

In [29]:
all_res_data = backup_all_res_data

In [30]:
scripts = []
for i in all_res_data:
    if len(i) > 1:
        scripts.append(i)
    else:
        pass
#All of the incomplete games in all_res_data will have a length of zero, so this is how we will filter out the unplayed games
#if necessary

In [31]:
script_list = []
for i in scripts:
    script_list.append(i[1])
#The [1] element is the one with all of the shot data

In [32]:
for i in range(0, len(script_list)):
    script_list[i] = str(script_list[i])
#turning all of our data into strings

In [33]:
data_list = []

# strip unnecessary symbols and get only JSON data 
for i in range(0, len(script_list)):
    ind_start = script_list[i].index("('")+2 
    ind_end = script_list[i].index("')") 
    json_data = script_list[i][ind_start:ind_end] 
    json_data = json_data.encode('utf8').decode('unicode_escape')

#convert string to json format
    data = json.loads(json_data)
    data_list.append(data)

In [34]:
all_data = []
match_id_list = []
x_list = []
y_list = []
home_score_list = []
away_score_list = []
player_list = []
xG_list = []
h_a_list = []
situation_list = []
result_list = []
team_list = []
opposing_team_list = []
minute_list = []
length = len(data_list)
for i in range(length):
    match_id = [] 
    x = []
    y = []
    home_score = []
    away_score = []
    player = []
    xG = []
    h_a = []
    situation = []
    result = []
    team = []
    opposing_team = []
    minute = []
    data_away = data_list[i]['a']
    data_home = data_list[i]['h']

    for index in range(len(data_home)):
        for key in data_home[index]:
            if key == 'X':
                x.append(data_home[index][key])
            if key == 'Y':
                y.append(data_home[index][key])
            if key == 'h_team':
                team.append(data_home[index][key])
            if key == 'a_team':
                opposing_team.append(data_home[index][key])
            if key == 'xG':
                xG.append(data_home[index][key])
            if key == 'situation':
                situation.append(data_home[index][key])
            if key == 'result':
                result.append(data_home[index][key])
            if key == 'h_goals':
                home_score.append(data_home[index][key])
            if key == 'a_goals':
                away_score.append(data_home[index][key])
            if key == 'minute':
                minute.append(data_home[index][key]) 
            if key == 'player':
                player.append(data_home[index][key])
            if key == 'match_id':
                match_id.append(data_home[index][key])
            if key == 'h_a':
                h_a.append(data_home[index][key])
        
        #NEED TO APPEND EVERYTHING TO ALL_DATA

    for index in range(len(data_away)):
        for key in data_away[index]:
            if key == 'X':
                x.append(data_away[index][key])
            if key == 'Y':
                y.append(data_away[index][key])
            if key == 'a_team':
                team.append(data_away[index][key])
            if key == 'h_team':
                opposing_team.append(data_away[index][key])
            if key == 'xG':
                xG.append(data_away[index][key])
            if key == 'situation':
                situation.append(data_away[index][key])
            if key == 'result':
                result.append(data_away[index][key])
            if key == 'h_goals':
                home_score.append(data_away[index][key])
            if key == 'a_goals':
                away_score.append(data_away[index][key])
            if key == 'minute':
                minute.append(data_away[index][key])
            if key == 'player':
                player.append(data_away[index][key])
            if key == 'match_id':
                match_id.append(data_away[index][key])
            if key == 'h_a':
                h_a.append(data_away[index][key]) 
                
    match_id_list.append(match_id)
    x_list.append(x)
    y_list.append(y)
    team_list.append(team)
    opposing_team_list.append(opposing_team)
    xG_list.append(xG)
    situation_list.append(situation)
    result_list.append(result)
    home_score_list.append(home_score)
    away_score_list.append(away_score)
    minute_list.append(minute)
    player_list.append(player)
    h_a_list.append(h_a)

In [35]:
match_id_list = [val for sublist in match_id_list for val in sublist]
x_list = [val for sublist in x_list for val in sublist]
y_list = [val for sublist in y_list for val in sublist]
team_list = [val for sublist in team_list for val in sublist]
opposing_team_list = [val for sublist in opposing_team_list for val in sublist]
xG_list = [val for sublist in xG_list for val in sublist]
situation_list = [val for sublist in situation_list for val in sublist]
result_list = [val for sublist in result_list for val in sublist]
home_score_list = [val for sublist in home_score_list for val in sublist]
away_score_list = [val for sublist in away_score_list for val in sublist]
minute_list = [val for sublist in minute_list for val in sublist]
player_list = [val for sublist in player_list for val in sublist]
h_a_list = [val for sublist in h_a_list for val in sublist]
#Flattening nested lists

In [36]:
col_names = ['match_id','x','y','xG','minute','situation','result','team','opposing_team','h_a','player','home_score','away_score']
df = pd.DataFrame([match_id_list,x_list,y_list,xG_list,minute_list,situation_list,result_list,team_list,opposing_team_list,h_a_list,player_list,home_score_list,away_score_list],index=col_names)
df = df.T
df = df.astype({'minute':'int'})
df = df.sort_values(by=['match_id','minute'])
df = df.reset_index()

In [37]:
df.isna().sum()
#Checking for null values

index            0
match_id         0
x                0
y                0
xG               0
minute           0
situation        0
result           0
team             0
opposing_team    0
h_a              0
player           0
home_score       0
away_score       0
dtype: int64

# xG

In [38]:
# test_home_list = np.where((df['result'] == 'Goal') & (df['h_a'] == 'h') | (df['result'] == 'OwnGoal') & (df['h_a'] == 'a'), 1, 0)

In [39]:
df['xG'] = df['xG'].astype(float)
df['match_id'] = df['match_id'].astype(int)

In [40]:
xg_total = df.groupby('team')['xG'].sum()
#Count total xG for every team

In [41]:
xg_total = pd.DataFrame(data=xg_total)
# xg_total = xg_total.reset_index(level=0)

In [42]:
xg_total['xGA'] = df.groupby('opposing_team')['xG'].sum()

In [43]:
xg_total['xG_difference'] = xg_total['xG'] - xg_total['xGA']

In [44]:
xg_total['matches_played'] = df.groupby('team')['match_id'].nunique()
#Get number of matches played for each team

In [45]:
xg_total = xg_total.reset_index(level=0)

In [46]:
xg_total['avg_xG_diff'] = round(xg_total['xG_difference'] / xg_total['matches_played'], 2)
xg_total['avg_xG_for'] = round(xg_total['xG'] / xg_total['matches_played'], 2)
xg_total['avg_xGA'] = round(xg_total['xGA'] / xg_total['matches_played'], 2)

In [47]:
xg_total.isna().sum()
#Checking for null values

team              0
xG                0
xGA               0
xG_difference     0
matches_played    0
avg_xG_diff       0
avg_xG_for        0
avg_xGA           0
dtype: int64

# Non Pen xG

In [48]:
non_pen_xg = df.groupby('team')
non_pen_xg = non_pen_xg.apply(lambda x: x[x['situation'] != 'Penalty']['xG'].sum())
#Getting non pen xG and adding to df
#Not working because this groupby result does not have a name so can't go into a df

In [49]:
non_pen_xga = df.groupby('opposing_team')
non_pen_xga = non_pen_xga.apply(lambda x: x[x['situation'] != 'Penalty']['xG'].sum())

In [50]:
non_pen_xga

opposing_team
Arsenal                    3.044849
Aston Villa                4.949908
Bournemouth                8.892331
Brentford                  6.077354
Brighton                   3.533493
Chelsea                    6.744414
Crystal Palace             6.972032
Everton                    7.246959
Fulham                     7.385355
Leeds                      4.813911
Leicester                  6.995191
Liverpool                  4.383746
Manchester City            2.674813
Manchester United          6.040331
Newcastle United           5.528709
Nottingham Forest          4.633676
Southampton                6.014566
Tottenham                  2.913505
West Ham                   5.267606
Wolverhampton Wanderers    2.705249
dtype: float64

In [51]:
xg_total['non_pen_xG'] = non_pen_xg.values
xg_total['non_pen_xGA'] = non_pen_xga.values

#Now need to match for team

In [52]:
xg_total['xG_difference'] = xg_total['xG'] - xg_total['xGA']

In [53]:
xg_total['non_pen_xg_difference'] = xg_total['non_pen_xG'] - xg_total['non_pen_xGA']

In [54]:
xg_total['avg_non_pen_xG_diff'] = round(xg_total['non_pen_xg_difference'] / xg_total['matches_played'], 2)
xg_total['avg_non_pen_xG_for'] = round(xg_total['non_pen_xG'] / xg_total['matches_played'], 2)
xg_total['avg_non_pen_xGA'] = round(xg_total['non_pen_xGA'] / xg_total['matches_played'], 2)

In [55]:
xg_total = xg_total.sort_values(by='avg_xG_diff', ascending=False).reset_index(drop=True)

In [56]:
xg_total

,team,xG,xGA,xG_difference,matches_played,avg_xG_diff,avg_xG_for,avg_xGA,non_pen_xG,non_pen_xGA,non_pen_xg_difference,avg_non_pen_xG_diff,avg_non_pen_xG_for,avg_non_pen_xGA
0,Manchester City,10.398971,2.674813,7.724158,4,1.93,2.60,0.67,9.637802,2.674813,6.962989,1.74,2.41,0.67
1,Arsenal,8.893193,3.044849,5.848344,4,1.46,2.22,0.76,8.893193,3.044849,5.848344,1.46,2.22,0.76
2,Liverpool,10.966730,5.144915,5.821814,4,1.46,2.74,1.29,10.966730,4.383746,6.582983,1.65,2.74,1.10
3,Brighton,6.806062,3.533493,3.272569,4,0.82,1.70,0.88,6.044893,3.533493,2.511400,0.63,1.51,0.88
4,Tottenham,4.688307,2.913505,1.774802,3,0.59,1.56,0.97,4.688307,2.913505,1.774802,0.59,1.56,0.97
5,Leeds,6.717046,4.813911,1.903135,4,0.48,1.68,1.20,6.717046,4.813911,1.903135,0.48,1.68,1.20
6,Manchester United,6.470684,6.040331,0.430353,4,0.11,1.62,1.51,6.470684,6.040331,0.430353,0.11,1.62,1.51
7,Brentford,6.407425,6.077354,0.330070,4,0.08,1.60,1.52,6.407425,6.077354,0.330070,0.08,1.60,1.52
8,Chelsea,6.204253,6.744414,-0.540161,4,-0.14,1.55,1.69,5.443084,6.744414,-1.301330,-0.33,1.36,1.69
9,Crystal Palace,6.344293,6.972032,-0.627739,4,-0.16,1.59,1.74,5.583124,6.972032,-1.388908,-0.35,1.40,1.74


In [57]:
# xg_total.to_excel("prem_xG_data.xlsx")

# In Home Games

# In Away Games

NEXT: Will add something to get all of the players from the match, and then count the defensive players to determine back 4 or back 5

Code will be -- take all of the elements where the class "player-position" is DC... get a count, if it's two then the team will be in a back four, if there are 3 then it's a back five with wingbacks